# Data Preprocessing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import cos, sin, arcsin, sqrt
from math import radians
from datetime import date
import holidays
from sklearn.preprocessing import MinMaxScaler

In [2]:
def read_and_preprocess(rides_file_path, stations_file_path):
    '''
    This function is used to read a rides file and all other necessary files and do the preprocessing.
    
    :param str rides_file_path: path to the rides file
    :param str stations_file_path: path to the stations file
    :return pd.DataFrame: dataframe ready to be used in the modelling process
    '''
    
    rides_df = pd.read_csv(rides_file_path, parse_dates=[0, 2])
    stations_df = pd.read_csv(stations_file_path)
    
    # turn all column names to lower case
    rides_df.columns = rides_df.columns.str.lower()
    stations_df.columns = stations_df.columns.str.lower()
    
    # this happens in file of 2021
    if not 'start_station_code' in rides_df.columns:
        rides_df = rides_df.rename(columns={'emplacement_pk_start': 'start_station_code',
                                            'emplacement_pk_end': 'end_station_code'})
    
    # this happens in file of August 2019, because of invalid station codes
    if rides_df['start_station_code'].dtype != 'int':
        
        # add column for integer values, insert None when a value can not be converted
        def to_int_or_none(val):
            try:
                return(int(val))
            except ValueError:
                return None
    
        rides_df['start_station_code_int'] = rides_df['start_station_code'].apply(to_int_or_none)
        rides_df['end_station_code_int'] = rides_df['end_station_code'].apply(to_int_or_none)
    
        # drop every row where station codes could not be converted to integer
        rides_df = rides_df.dropna()
        rides_df['start_station_code'] = rides_df['start_station_code_int'].astype('int')
        rides_df['end_station_code'] = rides_df['end_station_code_int'].astype('int')
        rides_df = rides_df.drop(columns=['start_station_code_int', 'end_station_code_int'])
    
    # aggregate rides: sum up rides between 0:00 to 12:00 and 12:00 to 0:00
    ride_counts_df = rides_df.groupby([pd.Grouper(key='start_date', freq='12h'), 'start_station_code'])['end_date'].count()
    ride_counts_df = ride_counts_df.to_frame()
    ride_counts_df = ride_counts_df.rename(columns={'end_date': 'count'})
    ride_counts_df = ride_counts_df.reset_index()
    
    # add am/pm flags (am = 0, pm = 1)
    ride_counts_df['pm'] = ride_counts_df['start_date'].dt.hour.map({0: 0, 12: 1})
    
    # join coordinates, elevation and density (1 km radius) of stations
    ride_counts_df = ride_counts_df.merge(
        stations_df[['code', 'latitude', 'longitude', 'density', 'elevation_meters']],
        left_on='start_station_code',
        right_on='code',
        # inner join removes any station not specified in the stations dataset
        how='inner'
    )
    # add total number of stations in this year
    stations_count = len(stations_df['code'].unique())
    ride_counts_df['stations_count'] = stations_count
    
    # add year, month and weekday
    ride_counts_df['year'] = ride_counts_df['start_date'].dt.year
    ride_counts_df['month'] = ride_counts_df['start_date'].dt.month
    ride_counts_df['weekday'] = ride_counts_df['start_date'].dt.weekday
    
    # add holiday flag
    ca_qc_holidays = holidays.country_holidays('CA', subdiv='QC')
    ride_counts_df['holiday'] = ride_counts_df['start_date'].apply(lambda d: d in ca_qc_holidays)
    
    # add weather data
    if ride_counts_df.loc[1, 'year'] < 2020:
        weather_df = pd.read_csv('data/Canadian_climate_history.csv', parse_dates=[0])
        weather_df.columns = weather_df.columns.str.lower()
        weather_df = weather_df[['local_date', 'mean_temperature_montreal', 'total_precipitation_montreal']]
        weather_df = weather_df.rename(
            columns={'mean_temperature_montreal': 'mean_temperature',
                     'total_precipitation_montreal': 'total_precipitation'}
        )
    # weather for 2020 and 2021
    else:
        weather_df = pd.read_csv('data/Weather_2020_2021.csv', parse_dates=[4])
        weather_df.columns = weather_df.columns.str.lower()
        weather_df = weather_df[["date/time", "mean temp (°c)", "total precip (mm)"]]
        weather_df = weather_df.rename(
            columns={'date/time': 'local_date',
                     'mean temp (°c)': 'mean_temperature',
                     'total precip (mm)': 'total_precipitation'}
        )
    # interpolate missing temperature and precipitation values
    weather_df[['mean_temperature', 'total_precipitation']] = weather_df[['mean_temperature', 'total_precipitation']].interpolate()
    # add date attribute to join on and join
    ride_counts_df['date'] = pd.to_datetime(ride_counts_df['start_date'].dt.date)
    ride_counts_df = ride_counts_df.merge(weather_df, left_on='date', right_on='local_date')
    
    # add distance to city center
    def haversine(row):
        lon1 = -73.56878
        lat1 = 45.50354
        lon2 = row['longitude']
        lat2 = row['latitude']
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * arcsin(sqrt(a)) 
        km = 6367 * c
        return km

    ride_counts_df['distance_to_center'] = ride_counts_df.apply(lambda row: haversine(row), axis=1)
    
    # only keep these columns
    ride_counts_df = ride_counts_df[[
        'latitude', 'longitude', 'distance_to_center',
        'year', 'month', 'weekday', 'pm', 'holiday',
        'mean_temperature', 'total_precipitation',
        'stations_count', 'elevation_meters', 'density',
        'count'
    ]]
    
    return ride_counts_df

In [ ]:
# 2014 - 2019 (all data)
# try reading file first
try:
    pd.read_csv('data/preprocessed_data/all.csv')
except FileNotFoundError:
    train_df = pd.DataFrame()
    for year in range(2014, 2019):
        stations_file_path = f'data/stations_preprocessed/Stations_{year}.csv'
        for month in range(4, 11):
            rides_file_path = f'data/{year}/OD_{year}-{month:02d}.csv'
            # run preprocessing function and append df
            month_df = read_and_preprocess(rides_file_path, stations_file_path)
            train_df = pd.concat([train_df, month_df]).reset_index(drop=True)
    # save file
    train_df.to_csv('data/preprocessed_data/all.csv', index=False)


In [3]:
# 2014 & 2017 (training data)
# try reading file first
try:
    pd.read_csv('data/preprocessed_data/train.csv')
except FileNotFoundError:
    train_df = pd.DataFrame()
    for year in range(2014, 2018):
        stations_file_path = f'data/stations_preprocessed/Stations_{year}.csv'
        for month in range(4, 11):
            rides_file_path = f'data/{year}/OD_{year}-{month:02d}.csv'
            # run preprocessing function and append df
            month_df = read_and_preprocess(rides_file_path, stations_file_path)
            train_df = pd.concat([train_df, month_df]).reset_index(drop=True)
    # save file
    train_df.to_csv('data/preprocessed_data/train.csv', index=False)

# 2018 (model validation data)
# try reading file first
try:
    pd.read_csv('data/preprocessed_data/validation.csv')
except FileNotFoundError:
    val_df = pd.DataFrame()
    stations_file_path = f'data/stations_preprocessed/Stations_2018.csv'
    for month in range(4, 11):
        rides_file_path = f'data/2018/OD_2018-{month:02d}.csv'
        # run preprocessing function and append df
        month_df = read_and_preprocess(rides_file_path, stations_file_path)
        val_df = pd.concat([val_df, month_df]).reset_index(drop=True)
    # save file
    val_df.to_csv('data/preprocessed_data/validation.csv', index=False)
    
# 2019 (test data)
try:
    pd.read_csv('data/preprocessed_data/test.csv')
except FileNotFoundError:
    test_df = pd.DataFrame()
    stations_file_path = f'data/stations_preprocessed/Stations_2019.csv'
    for month in range(4, 11):
        rides_file_path = f'data/2019/OD_2019-{month:02d}.csv'
        # run preprocessing function and append df
        month_df = read_and_preprocess(rides_file_path, stations_file_path)
        test_df = pd.concat([test_df, month_df]).reset_index(drop=True)
    # save file
    test_df.to_csv('data/preprocessed_data/test.csv', index=False)
    
# 2020 & 2021 (Corona years)
try:
    pd.read_csv('data/preprocessed_data/corona.csv')
except FileNotFoundError:
    corona_df = pd.DataFrame()
    for year in range(2020, 2021): ### just 2020 for now ###
        stations_file_path = f'data/stations_preprocessed/Stations_{year}.csv'
        for month in range(4, 11):
            rides_file_path = f'data/{year}/OD_{year}-{month:02d}.csv'
            # run preprocessing function and append df
            month_df = read_and_preprocess(rides_file_path, stations_file_path)
            corona_df = pd.concat([corona_df, month_df]).reset_index(drop=True)
    # save file
    corona_df.to_csv('data/preprocessed_data/corona.csv', index=False)

In [24]:
train = pd.read_csv('data/preprocessed_data/train.csv')
test = pd.read_csv('data/preprocessed_data/test.csv')
validation = pd.read_csv('data/preprocessed_data/validation.csv')
corona = pd.read_csv('data/preprocessed_data/corona.csv')

In [25]:
scaler = MinMaxScaler()

In [26]:
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)
validation_scaled = scaler.transform(validation)
corona_scaled = scaler.transform(corona)

In [27]:
train = pd.DataFrame(train_scaled, columns=train.columns)
test = pd.DataFrame(test_scaled, columns=test.columns)
validation = pd.DataFrame(validation_scaled, columns=validation.columns)
corona = pd.DataFrame(corona_scaled, columns=corona.columns)

In [28]:
train.to_csv('data/scaled_preprocessed_data/train.csv', index=False)
test.to_csv('data/scaled_preprocessed_data/test.csv', index=False)
validation.to_csv('data/scaled_preprocessed_data/validation.csv', index=False)
corona.to_csv('data/scaled_preprocessed_data/corona.csv', index=False)